In [99]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import math
from scipy.constants import N_A

In [100]:
Target_cell_number = 2e5
well_size = 125e-6
t_end = 60*60*5
t = np.geomspace(1e-10, t_end, 500)
t_span = [1e-10, t_end]
z0 = [0, 0]
tumour_cell_radius = 8e-6
tumour_cell_surface_area = 4*math.pi*((tumour_cell_radius)**2)

In [101]:
def model_S_monospecific(t, z, Ainit, rtot, kon, k2, koff):
     k = Ainit*kon
     Atot = well_size*N_A*Ainit/Target_cell_number
     A0 = Atot - z[0] - z[1] 
     Ag = rtot - z[0] - 2*z[1]

     dA10 = 2*(k*Ag*A0/Atot) - koff*z[0] - (k2*Ag*z[0]) + 2*koff*z[1]
     dA11 = (k2*Ag*z[0]) - 2*koff*z[1]

     return [dA10, dA11]

def monovalent_binding(t, z, Ainit, rtot, kon, koff):
     k = Ainit*kon
     Atot = well_size*N_A*Ainit/Target_cell_number
     A0 = Atot - z
     Ag = rtot - z

     dA1 = k*Ag*A0/Atot - koff*z

     return dA1

def EC50_finder(array, A0s):
    half_max = 0.5*np.max(array) 
    half_max_array = half_max*np.ones_like(array)
    indicies = np.argwhere(np.diff(np.sign(half_max_array-array)))
    return A0s[indicies[0]]

In [102]:
A0s = np.geomspace(1e-12, 1e-3, 500)

# Desired KDs: 1nm 10nm 100nm 1000nm
kons = [10**(4.5), 10**4, 10**(3.5), 10**3]
koffs = [10**(-4.5), 10**(-4), 10**(-3.5), 10**(-3)]

bivalent = np.zeros((len(kons), len(A0s)))
monovalent = np.zeros((len(kons), len(A0s)))

D = 1e-14
k2 = 4*D/tumour_cell_surface_area
k1 = kons[0]*Target_cell_number/(well_size*N_A)
rtot = 1e3

print(k2/k1)

591971.8742834312


In [103]:

for j, kon in enumerate(kons):
    koff = koffs[j]
    for i, Ainit in enumerate(A0s):

        # simulate monovalent binding

        z = solve_ivp(monovalent_binding, t_span, [0], method='Radau', t_eval=t, args=(Ainit, rtot, kon, koff)) 
        bound_ab = z.y[0]
        monovalent[j, i] = bound_ab[-1]

        # simulate bivalent binding
    
        z = solve_ivp(model_S_monospecific, t_span, z0, method='Radau', t_eval=t, args=(Ainit, rtot, kon, k2, koff))
        A1 = z.y[0]
        A2 = z.y[1]
        bivalent[j, i] = A1[-1] + A2[-1]
    print(EC50_finder(monovalent[j, :], A0s), EC50_finder(bivalent[j, :], A0s))


[9.86252179e-10] [4.86827104e-10]
[9.68215306e-09] [4.77923865e-09]
[9.90813657e-08] [4.69183451e-08]
[9.72693362e-07] [4.41865996e-07]


In [104]:
sns.set_theme()
%matplotlib qt
fig, ax = plt.subplots(1, 4, figsize=(40, 4.8))
ax[0].semilogx(A0s, monovalent[0, :], c='red', label='monovalent')
ax[0].semilogx(A0s, bivalent[0, :], c='blue', label = 'bivalent')
ax[1].semilogx(A0s, monovalent[1, :], c='red', label='monovalent')
ax[1].semilogx(A0s, bivalent[1, :], c='blue', label = 'bivalent')
ax[2].semilogx(A0s, monovalent[2, :], c='red', label='monovalent')
ax[2].semilogx(A0s, bivalent[2, :], c='blue', label = 'bivalent')
ax[3].semilogx(A0s, monovalent[3, :], c='red', label='monovalent')
ax[3].semilogx(A0s, bivalent[3, :], c='blue', label = 'bivalent')

ax[0].set_xlabel(r'$A^{init}$')
ax[1].set_xlabel(r'$A^{init}$')
ax[2].set_xlabel(r'$A^{init}$')
ax[3].set_xlabel(r'$A^{init}$')

ax[0].set_ylabel(r'# of bound Abs')

ax[0].set_ylim(0, 1000)
ax[1].set_ylim(0, 1000)
ax[2].set_ylim(0, 1000)
ax[3].set_ylim(0, 1000)

ax[0].set_title(r'$K_D = 1$ nM')
ax[1].set_title(r'$K_D = 10$ nM')
ax[2].set_title(r'$K_D = 100$ nM')
ax[3].set_title(r'$K_D = 1000$ nM')
plt.legend()
plt.show()


In [105]:
kon_powers = np.linspace(5, 3, 50)
koff_powers = np.linspace(-5, -3, 50)

kons = 10**kon_powers
koffs = 10**koff_powers

mono_Ec50s = np.zeros_like(kon_powers)
biv_Ec50s = np.zeros_like(kon_powers)

for i, kon in enumerate(kons):
    monovalent = np.zeros_like(A0s)
    bivalent = np.zeros_like(A0s)
    koff = koffs[i]
    for j, Ainit in enumerate(A0s):

        # simulate monovalent binding

        z = solve_ivp(monovalent_binding, t_span, [0], method='Radau', t_eval=t, args=(Ainit, rtot, kon, koff)) 
        bound_ab = z.y[0]
        monovalent[j] = bound_ab[-1]

        # simulate bivalent binding
    
        z = solve_ivp(model_S_monospecific, t_span, z0, method='Radau', t_eval=t, args=(Ainit, rtot, kon, k2, koff))
        A1 = z.y[0]
        A2 = z.y[1]
        bivalent[j] = A1[-1] + 2*A2[-1]
    
    mono_Ec50s[i] = EC50_finder(monovalent, A0s)
    biv_Ec50s[i] = EC50_finder(bivalent, A0s)



In [108]:
Kds = koffs/kons
sns.set_context('talk')
plt.loglog(Kds, mono_Ec50s, label = r'Monovalent binidng')
plt.loglog(Kds, biv_Ec50s, label = r'Bivalent binding')
plt.xlabel(r'$K_D$ (M)')
plt.ylabel(r'$Ec50$ (M)')
plt.legend(loc='best')
plt.show()